A discrete linear quantum walk is a quantum analog of a classical random walk, but it operates on a quantum system and exhibits quantum properties such as superposition and interference.

*Superposition:*

In a quantum walk, the walker can be in a superposition of multiple positions simultaneously. This is unlike a classical random walk, where the walker can only be in one position at a time.

*Interference:*

The probability amplitudes of the quantum walk can interfere with each other, leading to constructive or destructive interference. This interference can significantly alter the probability distribution of the walker's position compared to a classical random walk.

*Unitary Evolution:*

The evolution of the quantum walk is governed by unitary operators, ensuring that the process is reversible and preserves the total probability.

**Components of a Discrete Linear Quantum Walk**

*Hilbert Space:*

The walker's position is represented in a Hilbert space, typically with basis states corresponding to different positions.

*Coin Operator:*

A coin operator (analogous to flipping a coin in a classical random walk) determines the direction of the walk. It creates a superposition of possible directions.

*Shift Operator:*

The shift operator moves the walker based on the outcome of the coin operator. It shifts the walker's position in the superposition space.

*Initial State:*

The walker starts in an initial quantum state, which is typically a superposition of different positions or a specific position.

![](quantum_walk.jpg)

In [3]:
from classiq import *

# Define the size of the quantum system
size = 4

# Function to prepare a qubit in the |-> state
@qfunc
def prepare_minus(x: QBit):
    X(x)
    H(x)

In [ ]:
# Oracle function to mark the solution state in a Grover search-like setup
@qfunc
def diffuser_oracle(aux: Output[QNum], x: QNum):
    aux ^= (x != 0)

# Function to implement the diffuser (or inversion about the mean) in Grover's algorithm
@qfunc
def zero_diffuser(x: QNum):
    aux = QNum('aux')
    allocate(1, aux)
    within_apply(
        compute=lambda: prepare_minus(aux),
        action=lambda: diffuser_oracle
    )

In [ ]:
# Function to apply the W operation (quantum walk step) for a given vertex i
def W_iteration(i: int, vertices: QNum, adjacent_vertices: QNum):
    prob = [0] * 16  # Initialize probability vector
    if i == 0:
        prob[i + 1] = 1.0  # If at the first vertex, transition to the next
    elif i == 15:
        prob[i - 1] = 1.0  # If at the last vertex, transition to the previous
    else:
        prob[i - 1] = prob[i + 1] = 0.5  # Otherwise, equal probability to move left or right
    
    print(f'State={i}, prob vec={prob}')
    
    control(ctrl=vertices == i,
            operand=lambda: within_apply(
                compute=lambda: inplace_prepare_state(probabilities=prob, bound=0.01, target=adjacent_vertices),
                action=lambda: zero_diffuser(adjacent_vertices)))

# Function to apply the W operator across all vertices
@qfunc
def W_operator(vertices: QNum, adjacent_vertices: QNum):
    for i in range(2**size):
        W_iteration(i, vertices, adjacent_vertices)

In [ ]:
# Oracle to identify edges in the graph, used for marking specific vertices
@qfunc
def edge_oracle(res: Output[QBit], vertices: QNum, adjacent_vertices: QNum):
    res |= ((vertices - adjacent_vertices) == 1) | ((adjacent_vertices - vertices) == 1)

In [ ]:
# Function to swap the states of two qubits
@qfunc
def bitwise_swap(x: QArray[QBit], y: QArray[QBit]):
    repeat(count=x.len,
           iteration=lambda i: SWAP(x[i], y[i]))

In [ ]:
# S operator function to apply the swap based on edge oracle
@qfunc
def S_operator(vertices: QNum, adjacent_vertices: QNum):
    res = QNum('res')
    edge_oracle(res, vertices, adjacent_vertices)
    control(ctrl=res == 1,
            operand=lambda: bitwise_swap(vertices, adjacent_vertices))

In [ ]:
# Main function to set up and execute the quantum walk and swap operations
@qfunc
def main(vertices: Output[QNum], adjacent_vertices: Output[QNum]):
    allocate(size, vertices)
    hadamard_transform(vertices)  # Initialize the vertices in superposition
    allocate(size, adjacent_vertices)

    W_operator(vertices, adjacent_vertices)  # Apply quantum walk operation
    S_operator(vertices, adjacent_vertices)  # Apply swap operation based on edge conditions

In [ ]:
# Create and synthesize the quantum model
qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)


The code simulates a discrete linear quantum walk on a 4-qubit system. It initializes the vertices in a superposition, applies quantum walk steps using controlled diffusion, and swaps states based on adjacency, mimicking the behavior of a classical random walk but leveraging quantum properties for potentially faster computation and different behavior due to superposition and interference.